In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [15]:
o2 = pd.read_csv(o2_path)
edw = pd.read_csv(edw_path)

## Fix Norepinephrine
- remove flag
- set to zero if na

In [17]:
edw = edw.drop(columns='Norepinephrine_flag')

In [18]:
edw['Norepinephrine_rate'] = [0 if pd.isna(x) else x for x in edw.Norepinephrine_rate]

In [19]:
edw.Norepinephrine_rate.isnull().value_counts()

False    16405
Name: Norepinephrine_rate, dtype: int64

Make sure annotations are aligned.

## Relabel preintubation after changes

In [20]:
edw['extubation_status'] = edw['extubation_status'].replace({'preintubation':'preintubated'})

In [27]:
preintub_days = []
for p in edw.Patient_id_2.unique():
    sub = edw[edw.Patient_id_2 == p]
    pt_day = sub.day.values[0]
    i = sub.patient_day.values[0]
    while pt_day <= sub.day.max() and sub[sub.patient_day == i].Intubation_flag.values[0] == 0 and \
    sub[sub.patient_day == i].extubation_status.values[0] != 'dni transplant':
        # if sub[sub.patient_day == i].extubation_status.values[0] != 'preintubated':
            preintub_days.append(i)
            pt_day += 1
            i = f"{p}/{pt_day}"

In [28]:
len(preintub_days)

1895

In [30]:
len(edw[edw.extubation_status == 'preintubated'])

3

These will be correct once I take out the missing in Epic days

In [31]:
edw = edw[edw.extubation_status != 'Missing in Epic- remove']

The days that used to be preintubated but now aren't should be considered extubated

In [32]:
edw.loc[edw.patient_day.isin(preintub_days),'extubation_status'] = 'preintubated'

In [33]:
edw[['extubation_status','Intubation_flag']].value_counts(dropna=False)

extubation_status    Intubation_flag
intubated            1                  12333
preintubated         0                   1892
extubated            0                   1516
dni transplant       0                    362
failed               0                    125
dni                  0                    109
same-day fail        0                     40
suspected dni        0                     13
ECMO non-successful  0                      9
early extubation     0                      3
dtype: int64

In [34]:
new_status_dict = {
    r.patient_day:r.extubation_status for i,r in edw.iterrows()
}
o2['extubation_status'] = o2['patient_day'].map(new_status_dict)

In [35]:
# o2.to_csv()
# edw.to_csv()

## Trim

In [36]:
edw.day_bucket_starts = pd.to_datetime(edw.day_bucket_starts)

In [39]:
trimmed_list = []

for patient in edw['Patient_id_2'].unique():
    sub = edw[edw['Patient_id_2'] == patient]
    labels = list(sub['Intubation_flag'][1:])
    next_day_ext_status = list(sub['extubation_status'][1:])
    features = sub.iloc[:-1,:]
    features['labels'] = labels
    features['next_day_extubation_status'] = next_day_ext_status
    trimmed_list.append(features)

trimmed_df = pd.concat(trimmed_list, ignore_index=False)

In [40]:
len(trimmed_df)

15462

For each patient stay, remove any preintubated days and take all intubated days leading up to the first extubation. If no extubation, take all intubated days.

In [41]:
filtered_list = []
for p in trimmed_df.Patient_id_2.unique():
    sub = trimmed_df[trimmed_df.Patient_id_2 == p]
    sub = sub[~sub.extubation_status.isin(['preintubation','preintubated'])]
    df_patient_index = sub[
    sub.next_day_extubation_status.isin(['extubated','early extubation','failed','same-day fail','ECMO non-successful'])].first_valid_index()  
    if df_patient_index is not None:
        df_patient = sub.loc[:df_patient_index]
        filtered_list.append(df_patient)
    elif len(sub[sub.next_day_extubation_status.isin(['dni','suspected dni','dni transplant'])]) > 0:
        df_patient_index = sub[
        sub.next_day_extubation_status.isin(['dni','suspected dni','dni transplant'])].first_valid_index()
        df_patient = sub.loc[:df_patient_index-1]
        filtered_list.append(df_patient)
    else:
        filtered_list.append(sub)
filtered_df = pd.concat(filtered_list, ignore_index=False)

In [42]:
filtered_df.extubation_status.value_counts()

intubated    9829
Name: extubation_status, dtype: int64

In [43]:
filtered_df.next_day_extubation_status.value_counts()

intubated              9323
extubated               392
failed                   76
same-day fail            34
early extubation          3
ECMO non-successful       1
Name: next_day_extubation_status, dtype: int64

In [44]:
filtered_df = filtered_df[filtered_df.next_day_extubation_status != 'ECMO non-successful']

## Split into train/val/test as well as fail and trach

In [45]:
trach_pts = filtered_df[filtered_df.trach_received == True].pt_study_id.unique()

In [46]:
len(trach_pts)

207

In [47]:
fail_pts = filtered_df[
(filtered_df.next_day_extubation_status.isin(['failed','same-day fail']))
& (~filtered_df.pt_study_id.isin(trach_pts))
].pt_study_id.unique()

In [48]:
len(fail_pts)

41

In [49]:
filtered_pts = list(trach_pts).copy()
filtered_pts.extend(fail_pts)
use_df = filtered_df[~filtered_df.pt_study_id.isin(filtered_pts)]

In [50]:
use_df.pt_study_id.nunique()

448

This is expectedly down from 579 from last iteration due to more flagged trach patients

In [51]:
pt_data = use_df.drop_duplicates(subset='pt_study_id')

In [52]:
pt_data.day_bucket_starts = pd.to_datetime(pt_data.day_bucket_starts)

In [58]:
new_train_val_patients = list(pt_data[pt_data.day_bucket_starts < pd.to_datetime(cutoff)].pt_study_id)
new_test_patients = list(pt_data[pt_data.day_bucket_starts >= pd.to_datetime(cutoff)].pt_study_id)

new_train_patients, new_val_patients = train_test_split(new_train_val_patients, test_size=0.2, random_state=42)

new_train_df = use_df[use_df.pt_study_id.isin(new_train_patients)]
new_val_df = use_df[use_df.pt_study_id.isin(new_val_patients)]
new_test_df = use_df[use_df.pt_study_id.isin(new_test_patients)]

In [59]:
new_train_df.pt_study_id.nunique()

288

In [60]:
new_val_df.pt_study_id.nunique()

72

In [61]:
new_test_df.pt_study_id.nunique()

88

In [62]:
new_fail_df = filtered_df[filtered_df.pt_study_id.isin(fail_pts)]

In [63]:
new_trach_df = filtered_df[filtered_df.pt_study_id.isin(trach_pts)]

## Format columns

In [64]:
format = pd.read_csv(format_path,index_col=0)

In [65]:
fill_cols = []
for col in format.columns:
    if col not in filtered_df.columns:
        print(col)
        fill_cols.append(col)

Norepinephrine_flag
old_extubation_status
Patient_id
patient
trach_collar_status


In [66]:
junk_cols = [x for x in filtered_df.columns if x.startswith('Unnamed: 0')]

In [67]:
drop_cols = []
for col in filtered_df.columns:
    if col not in format.columns and col not in junk_cols and col != 'Minute_Ventilation':
        print(col)
        drop_cols.append(col)

pt_study_id
ir_id
visit_key
day_bucket_starts
day_bucket_ends
SOFA_score
Mean_arterial_pressure
PaO2FIO2_ratio
calendar_day
patient_calendar_day
death_date
discharge_disposition_name
binary_outcome
old_id
old_Patient_id_2
trach_received


Save each dataframe with and without additional info. Drop all except Minute_ventilation for Alec's scripts

In [70]:
df_dict = {
    'train':new_train_df,
    'val':new_val_df,
    'test':new_test_df,
    'fail':new_fail_df,
    'trach':new_trach_df
}

In [71]:
for ds in df_dict.keys():
    df = df_dict[ds]
    df = df.drop(columns=junk_cols)
    for col in fill_cols:
        df[col] = ''
    df.to_csv(f"{outdir}/{ds}_additional_data.csv")
    df = df.drop(columns=drop_cols)
    df.to_csv(f"{outdir}/{ds}.csv")